In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!mkdir -p "/content/project"

In [8]:
import requests
import os
import time
from urllib.request import urlretrieve
import zipfile
import json

# Constants
CLIENT_ID = "__QBSpz6W2EwgMOMql4esPIccKQH9MEOANgSfd3RNEU"
SEASON = "summer"
IMAGE_COUNT = {"train": 6000, "validation": 1500, "test": 1500}
PER_PAGE = 30
BASE_PATH = "/content/project"
SECONDS_BETWEEN_REQUESTS = 72  # Throttle requests to stay under rate limit
PROGRESS_FILE = "/content/download_progress.json"

In [9]:
# Function to fetch images
def get_unsplash_photos(keyword, client_id, page, per_page=PER_PAGE):
    url = "https://api.unsplash.com/search/photos"
    params = {
        "query": keyword,
        "client_id": client_id,
        "per_page": per_page,
        "page": page
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return [photo['urls']['regular'] for photo in response.json()['results']]
    else:
        print(f"Failed to fetch photos for {keyword}, page {page}: {response.status_code}")
        return []

# Function to download and save images with custom filenames
def download_and_save_images(image_urls, folder_path, image_name, start_index):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    for i, url in enumerate(image_urls, start=start_index):
        filename = f"{folder_path}/{image_name}_{i}.jpg"
        urlretrieve(url, filename)

# Function to download images for a season and dataset part
def download_season_images(season, part, target_count, progress):
    folder_path = os.path.join(BASE_PATH, season, part)
    current_page = progress.get(season, {}).get(part, 1)
    downloaded_count = (current_page - 1) * PER_PAGE
    start_index = downloaded_count

    while downloaded_count < target_count:
        image_urls = get_unsplash_photos(season, CLIENT_ID, current_page)
        download_and_save_images(image_urls, folder_path, season.capitalize(), start_index)
        print(f"Successfully download photos on page {current_page}")
        downloaded_count += len(image_urls)
        current_page += 1
        start_index += len(image_urls)
        save_progress(season, part, current_page)
        if downloaded_count < target_count:
            time.sleep(SECONDS_BETWEEN_REQUESTS)  # Throttle requests

# Save progress to a file
def save_progress(season, part, page):
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as file:
            progress = json.load(file)
    else:
        progress = {}

    if season not in progress:
        progress[season] = {}
    progress[season][part] = page

    with open(PROGRESS_FILE, 'w') as file:
        json.dump(progress, file)

# Load progress from a file
def load_progress():
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as file:
            return json.load(file)
    return {}

# Zip the folders
def zip_folders():
    with zipfile.ZipFile('/content/project.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(BASE_PATH):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), BASE_PATH))

In [10]:
# Main download loop with progress tracking
progress = load_progress()
for part, count in IMAGE_COUNT.items():
    download_season_images(SEASON, part, count, progress)

zip_folders()
print("Zipping complete. Download '/content/project.zip'")

Successfully download photos on page 1
Successfully download photos on page 2
Successfully download photos on page 3
Successfully download photos on page 4
Successfully download photos on page 5
Successfully download photos on page 6
Successfully download photos on page 7
Successfully download photos on page 8
Successfully download photos on page 9
Successfully download photos on page 10
Successfully download photos on page 11
Successfully download photos on page 12
Successfully download photos on page 13
Successfully download photos on page 14
Successfully download photos on page 15
Successfully download photos on page 16
Successfully download photos on page 17
Successfully download photos on page 18
Successfully download photos on page 19
Successfully download photos on page 20
Successfully download photos on page 21
Successfully download photos on page 22
Successfully download photos on page 23
Successfully download photos on page 24
Successfully download photos on page 25
Successfu

ConnectionError: ignored